# Import libraries

In [2]:
# Load Libraries
import sys
import os
sys.path.append(os.path.abspath("../"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

from functions.pyflood_functions import *
from functions.pyflood_calibration_attenuation_functions import *

# Import paths and settings from config
from config import (
    DEM_FILE,
    ALIGNED_WATER_STATIONS_SHP,
    SEA_CORE_TIF,
    CFM_WD_TIF,
    CFM_WL_TIF,
    THRESHOLD_DEPTH,
    DISTANCE_TO_WATER_BODY,
    LANDCOVER_FILE,
    ALIGNED_HWM_SHP,
    BAYESIAN_OPTIMIZATION_CSV,
    MANUAL_REDUCTION_FACTORS,
    CFM_A_WD_TIF,
    CFM_A_WL_TIF,
    CFM_A_WD_FINAL_TIF,
    CFM_A_WL_FINAL_TIF
)

## Kriging with External Drift Process and Coastal Flood Map (CFM) Generation

In [4]:
# --------------------------------------------------
# Load DEM and Aligned Stations
# --------------------------------------------------
z_dem, transform, crs = load_raster_data(DEM_FILE)
dem_lon, dem_lat, _ = extract_raster_coordinates(z_dem, transform, crs)

aligned_stations = load_aligned_water_stations(ALIGNED_WATER_STATIONS_SHP)
lon_stations = aligned_stations[:, 0]
lat_stations = aligned_stations[:, 1]
wl_stations = aligned_stations[:, 2]
dem_stations = aligned_stations[:, 3]

print(f"DEM size: {z_dem.shape}, Valid elevations: {np.count_nonzero(~np.isnan(z_dem)):,}")

# --------------------------------------------------
# Kriging Interpolation of Water Levels
# --------------------------------------------------
wl_pred, wl_var = krige_water_levels(
    lon_stations, lat_stations, wl_stations, dem_stations,
    dem_lon, dem_lat, z_dem
)
# Optional: Save raw predicted water levels and associated variance
# save_raster(wl_pred, reference_raster=DEM_FILE, output_filename="../output_data/wl_pred.tif")
# save_raster(wl_var, reference_raster=DEM_FILE, output_filename="../output_data/wl_var.tif")

Coordinates extracted in UTM.
Shapefile fields available: ['X_UTM', 'Y_UTM', 'Water_Leve', 'DEM_Elevat', 'geometry']
Loaded 71 aligned water stations.
DEM size: (24000, 24000), Valid elevations: 576,000,000
Total number of prediction points: 576000000
[Before kriging] Memory Usage: 13.12 GB | CPU Usage: 1.0%
Parallel Kriging completed in 563.09 seconds.
[After kriging] Memory Usage: 16.95 GB | CPU Usage: 0.9%


In [5]:
# --------------------------------------------------
# Compute Water Depth and Save Initial Predictions
# --------------------------------------------------
wd_pred = compute_water_depth(wl_pred, z_dem)
save_raster(wd_pred, reference_raster=DEM_FILE, output_filename="../output_data/wd_pred.tif")

# --------------------------------------------------
# Mask with Sea Core and Save Coastal Flood Maps (CFM)
# --------------------------------------------------
wd_pred_masked = mask_with_sea_core(wd_pred, SEA_CORE_TIF)
save_raster(wd_pred_masked, reference_raster=DEM_FILE, output_filename=CFM_WD_TIF)

wl_pred_masked = wd_pred_masked + z_dem
save_raster(wl_pred_masked, reference_raster=DEM_FILE, output_filename=CFM_WL_TIF)

apply_threshold_and_export(
    input_file=CFM_WD_TIF,
    threshold_value=THRESHOLD_DEPTH,
    output_suffix="_over_1m"
)

Saved raster to: ../output_data/wd_pred.tif
Saved raster to: ../output_data\CFM_Flood_Water_Depth.tif
Saved raster to: ../output_data\CFM_Flood_Water_Level.tif
Thresholded raster exported to: ../output_data\CFM_Flood_Water_Depth_over_1m.tif


## Calibration Process and Coastal Flood Map + Attenuation (CFM+A) Generation

In [7]:
!pip install bayesian-optimization

In [8]:
# --------------------------------------------------
# Load Land Cover, Distance to Water Body, and Aligned HWMs
# --------------------------------------------------
rf_test, _, _ = load_raster_data(LANDCOVER_FILE)
distances, _, _ = load_raster_data(DISTANCE_TO_WATER_BODY)
gdf_aligned = gpd.read_file(ALIGNED_HWM_SHP)

# --------------------------------------------------
# Validate CFM Predictions Against Observed HWMs
# --------------------------------------------------
gdf_comparison = compare_hwms_to_model(gdf_aligned, wl_pred_masked, crs)

observed_hwm = gdf_comparison["hwm"].values
predicted_wl = gdf_comparison["z_wl"].values
rmse, mae, scatter_index, bias, num_points = compute_error_metrics(observed_hwm, predicted_wl)
print(f"Initial RMSE: {rmse:.3f} m | MAE: {mae:.3f} m | Scatter Index: {scatter_index:.3f} | Bias: {bias:.3f} m")

initial_flood_extent_points = num_points
print(f"Total of HWMs within Flood Extent of CFM: {initial_flood_extent_points}")

# --------------------------------------------------
# Bayesian Optimization of Reduction Factors
# --------------------------------------------------
# Define reduction factor bounds (min and max dictionaries)

value_mapping_min = {
2.0: 0.00025,
3.0: 0.000139,
4.0: 0.000104,
5.0: 0.000063,
6.0: 0.00016,
7.0: 0.000052,
8.0: 0.000052,
9.0: 0.000156,
10.0: 0.000125,
11.0: 0.0001,
12.0: 0.000096,
13.0: 0.000041,
14.0: 0.000088,
15.0: 0.000088,
16.0: 0.000041,
17.0: 0.000088,
18.0: 0.000088,
19.0: 0.000053,
20.0: 0.000053,
21.0: 0.000038,
22.0: 0.000038,
23.0: 0.000038
}

value_mapping_max = {
2.0: 0.001,
3.0: 0.000667,
4.0: 0.0005,
5.0: 0.00025,
6.0: 0.0004,
7.0: 0.00025,
8.0: 0.00025,
9.0: 0.0005,
10.0: 0.0004,
11.0: 0.0004,
12.0: 0.00035,
13.0: 0.000225,
14.0: 0.00025,
15.0: 0.00025,
16.0: 0.000225,
17.0: 0.00025,
18.0: 0.00025,
19.0: 0.000115,
20.0: 0.000115,
21.0: 0.000125,
22.0: 0.000125,
23.0: 0.000125
}

pbounds = define_parameter_bounds(value_mapping_min, value_mapping_max)

if not os.path.exists(BAYESIAN_OPTIMIZATION_CSV):
    pd.DataFrame(columns=['Iteration', 'MSE', 'Num_Valid_Points'] + [f'rf_{int(k)}' for k in value_mapping_min.keys()]).to_csv(BAYESIAN_OPTIMIZATION_CSV, index=False)

from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(
    f=lambda **rf_values: objective_function_bayesopt(
        rf_values, rf_test, distances, wl_pred, z_dem, gdf_aligned, initial_flood_extent_points, BAYESIAN_OPTIMIZATION_CSV
    ),
    pbounds=pbounds
)

optimizer.maximize(init_points=22, n_iter=10)

print("Best reduction factors found:", optimizer.max)

# --------------------------------------------------
# Apply Optimized Reduction Factors
# --------------------------------------------------
best_rf = optimizer.max['params']
best_rf_fixed = {int(k.split('_')[1]): v for k, v in best_rf.items()}

reduced_wl_best, reduced_wd_best = apply_optimized_reduction_factors(
    rf_test, distances, best_rf_fixed, wl_pred_masked, z_dem
)

reduced_wl_best_fixed = np.full_like(reduced_wl_best, np.nan)
valid_mask = ~np.isnan(reduced_wd_best)
reduced_wl_best_fixed[valid_mask] = reduced_wd_best[valid_mask] + z_dem[valid_mask]
reduced_wl_best = reduced_wl_best_fixed

reduced_wl_best_masked = mask_attenuated_results(reduced_wl_best, SEA_CORE_TIF)
reduced_wd_best_masked = mask_attenuated_results(reduced_wd_best, SEA_CORE_TIF)

save_attenuated_raster(reduced_wd_best_masked, DEM_FILE, CFM_A_WD_TIF)
save_attenuated_raster(reduced_wl_best_masked, DEM_FILE, CFM_A_WL_TIF)

apply_threshold_and_export(
    input_file=CFM_A_WD_TIF,
    threshold_value=THRESHOLD_DEPTH,
    output_suffix="_over_1m"
)

Comparison results saved to: hwm_vs_wl_comparison.shp
Initial RMSE: 0.520 m | MAE: 0.371 m | Scatter Index: 0.146 | Bias: 0.179 m
Total of HWMs within Flood Extent of CFM: 57
|   iter    |  target   |   rf_10   |   rf_11   |   rf_12   |   rf_13   |   rf_14   |   rf_15   |   rf_16   |   rf_17   |   rf_18   |   rf_19   |   rf_2    |   rf_20   |   rf_21   |   rf_22   |   rf_23   |   rf_3    |   rf_4    |   rf_5    |   rf_6    |   rf_7    |   rf_8    |   rf_9    |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Iteration 0: Modified MSE = 0.262724, Num_Valid_Points = 57
| 1         | -0.2627   | 0.000282  | 0.000285  | 0.0001971 | 0.0001511 | 0.0001218 | 0.0001385 | 0.000139  | 0.0001833 | 0.000235  | 7.61e-05  | 0.0002721 | 0.0001016 | 9.363e-05 | 8.2

In [9]:
# --------------------------------------------------
# Apply Manual Reduction Factors (Defined in config.py)
# --------------------------------------------------
reduced_wl_final, reduced_wd_final = apply_optimized_reduction_factors(
    rf_test, distances, MANUAL_REDUCTION_FACTORS, wl_pred_masked, z_dem
)

reduced_wl_final_fixed = np.full_like(reduced_wl_final, np.nan)
valid_mask = ~np.isnan(reduced_wd_final)
reduced_wl_final_fixed[valid_mask] = reduced_wd_final[valid_mask] + z_dem[valid_mask]
reduced_wl_final = reduced_wl_final_fixed

reduced_wl_final_masked = mask_attenuated_results(reduced_wl_final, SEA_CORE_TIF)
reduced_wd_final_masked = mask_attenuated_results(reduced_wd_final, SEA_CORE_TIF)

save_attenuated_raster(reduced_wd_final_masked, DEM_FILE, CFM_A_WD_FINAL_TIF)
save_attenuated_raster(reduced_wl_final_masked, DEM_FILE, CFM_A_WL_FINAL_TIF)

apply_threshold_and_export(
    input_file=CFM_A_WD_FINAL_TIF,
    threshold_value=THRESHOLD_DEPTH,
    output_suffix="_over_1m"
)

Saved attenuated raster to: ../output_data\CFM_A_Flood_Water_Depth_FINAL.tif
Saved attenuated raster to: ../output_data\CFM_A_Flood_Water_Level_FINAL.tif
Thresholded raster exported to: ../output_data\CFM_A_Flood_Water_Depth_FINAL_over_1m.tif
